# Analyse 1 : utilisation de regexp et de patterns spacy

on essaie d'extraire des termes connus, et des locutions typiques, pour détecter les références aux personnages et aux lieux.



## Trouver et lister les fichiers

In [1]:
from glob import glob
import os.path

In [2]:
# FIXME fournir code lecture fichiers depuis Google Drive ici (archive format zip)

In [3]:
PATH_TEXT_FILES_DIR = "/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts"

In [4]:
files = sorted(glob(os.path.join(PATH_TEXT_FILES_DIR, "*.txt")))
print("Found", len(files), "files.")
files[:10]

Found 100 files.


['/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00101_Adam.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00102_Adam.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00201_Audoux.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00301_Aimard.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00302_Aimard.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00401_Allais.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00501_Balzac.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00502_Balzac.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00503_Balzac.txt',
 '/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA00601_Boisgobey.txt']

In [5]:
sample = files[50]
sample

'/home/jchazalo/tmp/French_ELTEC_NER_Open_Dataset/texts/tr_FRA03001_Ohnet.txt'

## Import spacy and start processing

In [6]:
import spacy


## L'objet [`Language`](https://spacy.io/api/language)
On construit une nouvelle chaîne de traitements de plusieurs façon. La manière la plus simple est de construire une chaîne de traitement vide (ou presque) pour le français à l'aide de la "fabrique" à chaînes de traitement `spacy.blank(LANGAGE)`.

In [7]:
nlp = spacy.blank("fr")


La chaîne de traitements contient différents traitements appliqués les uns après les autres.
On peut afficher cette liste de traitements à l'aide de l'attribut `pipe_names` de l'objet `nlp`.

In [8]:
nlp.pipe_names

[]

Par défaut, une chaîne de traitement ne contient rien… Sauf un *tokenizer*, d'où l'importance de préciser la langue !

## L'objet [`Doc`](https://spacy.io/api/doc)
On obtient un objet [`Doc`](https://spacy.io/api/doc) en appliquant la chaîne de traitement [`Language`](https://spacy.io/api/language) à une chaîne de texte.

Cet objet [`Doc`](https://spacy.io/api/doc) est central pour Spacy car va être progressivement enrichi par chacun de traitements qui va venir y piocher les informations dont il a besoin en entrée, et ajouter les informations qu'il a calculées.
Par exemple, le composant "ner" va venir affecter une étiquette (*"label"*) à chacun des *tokens* du document. Il va stocker cette information dans un nouvel attribut `doc.ents` du document.

L'attribut `doc.text` quant à lui contient la liste des *tokens* extraits.

![](https://spacy.io/images/architecture.svg)

![](https://spacy.io/images/pipeline.svg)

In [9]:
# Créé en traitant une chaine de caractères avec l'objet nlp
doc = nlp("Bonjour tout le monde !")

On peut parcourir les *tokens* extraits d'un [`Doc`](https://spacy.io/api/doc) à l'aide d'une boucle classique en Python.

In [10]:
# Itère sur les tokens dans un Doc
for token in doc:
    print(token.text)

Bonjour
tout
le
monde
!


In [11]:
# On peut sélectionner un token particulier, grâce à son indice dans le document
token = doc[1]
token

tout

### Exemple de cellule avec contenu caché par défaut
#### 🚧 <b>Essayez à présent de sélectionner et afficher les <i>tokens</i> "tout le monde".</b>

<details>
<summary>Indices</summary>

Vous pouvez utiliser les *ranges* pour sélectionner plusieurs éléments d'un itérable. Voici un exemple de la syntaxe à utiliser :
```python
ma_liste = [0, 1, 2, 3]
print(ma_liste[1:3])
```
</details>

<details>
<summary>Solution</summary>

On applique cette syntaxe pour sélectionner les tokens du rang 1 (2e token, inclus) au rang 4 (non inclus) :

```python
doc[1:4]
```
</details>


In [12]:
# On peut également utiliser les "ranges" Python pour sélectionner plusieurs tokens
span = doc[1:4]
span

tout le monde

In [13]:
# On peut également accéder aux attributs d'un span
span.text

'tout le monde'

## 

In [15]:
## Autres attributs des tokens et des spans

doc = nlp("Cela coûte 5 €.")

print("Index :   ", [token.i for token in doc])
print("Text :    ", [token.text for token in doc])

print("is_alpha :", [token.is_alpha for token in doc])
print("is_punct :", [token.is_punct for token in doc])
print("like_num :", [token.like_num for token in doc])

Index :    [0, 1, 2, 3, 4]
Text :     ['Cela', 'coûte', '5', '€', '.']
is_alpha : [True, True, False, False, False]
is_punct : [False, False, False, False, True]
like_num : [False, False, True, False, False]


## Utilisation d'un pipeline avec un pos_tagger et un reconnaisseur d'entités nommées

In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [ ]:
nlp = spacy.load("fr_core_news_sm")
# Charge un vocabulaire français (utilisé pour la tokenization), et bien d'autres composants !
nlp.pipe_names

In [37]:
# Process a text
doc = nlp("La journée de formation à Lyon se déroule bien.")
for token in doc:
    # print(token.text, token.pos_, token.dep_, token.head.text)
    print(f"{token.text:>10s}", f"{token.pos_:>6s}", f"{token.dep_:>12s}", f"{token.head.text:>10s}")

        La    DET          det    journée
   journée   NOUN        nsubj    déroule
        de    ADP         case  formation
 formation   NOUN         nmod    journée
         à    ADP         case       Lyon
      Lyon  PROPN         nmod    journée
        se   PRON    expl:comp    déroule
   déroule   VERB         ROOT    déroule
      bien    ADV       advmod    déroule
         .  PUNCT        punct    déroule


## Visualisation des résultats avec *displaCy*

In [38]:
from spacy import displacy

In [39]:
displacy.render(doc, style="ent", jupyter=True)

In [42]:
displacy.render(doc, style="span", jupyter=True)

In [43]:
displacy.render(doc, style="dep", jupyter=True)

## Utilisation de la nouvelle chaîne de traitement pour traiter nos données

In [21]:
def load_text(filename: str) -> str:
    with open(filename, encoding="utf8") as in_file:
        return "".join(in_file.readlines())

In [22]:
text = load_text(files[50])
print(text)
doc = nlp(text)

I
Dans un de ces charmants chemins creux de Normandie, serpentant entre les levées, plantées de grands arbres, qui entourent les fermes d'un rempart de verdure impénétrable au vent et au soleil, par une belle matinée d'été, une amazone, montée sur une jument de forme assez médiocre, s'avançait au pas, les rênes abandonnées, rêveuse, respirant l'air tiède, embaumé du parfum des trèfles en fleurs. Avec son chapeau de feutre noir entouré d'un voile de gaze blanche, son costume de drap gris fer à longue jupe, elle avait fière tournure. On eût dit une de ces aventureuses grandes dames qui, au temps de Stofflet et de Cathelineau, suivaient hardiment l'armée royaliste, dans les traînes du Bocage, et éclairaient de leur sourire la sombre épopée vendéenne.
Élégante et svelte, elle se laissait aller gracieusement au mouvement de sa monture, fouettant distraitement de sa cravache les tiges vertes des genêts. Un lévrier d'Écosse au poil rude et rougeâtre l'accompagnait, réglant son allure souple s

In [23]:
# Itère sur les entités prédites
for ent in doc.ents:
    # Affiche le texte de l'entité et son label
    print(ent.text, ent.label_)

I LOC
Normandie LOC
Stofflet PER
Cathelineau LOC
Bocage LOC
Élégante ORG
Écosse LOC
amazone LOC
Madame PER
Rassurez PER
Monsieur PER
Allons PER
Fox ORG
Où MISC
La Neuville LOC
Madame PER


In [24]:
spacy.explain("GPE")

'Countries, cities, states'

In [25]:
spacy.explain("LOC")

'Non-GPE locations, mountain ranges, bodies of water'

## (❓ opt ❓) Utilisation du matcher Spacy
TODO montrer comment filtrer sur lexème, nature ou fonction du token.

In [26]:
pattern = [{"POS": "PROPN"}]

In [27]:
spacy.explain("PROPN")

'proper noun'

In [28]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
matcher.add("NOMS_PROPRES", [pattern])
matches = matcher(doc)

In [29]:
print("Nombre de correspondances trouvées :", len(matches))

# Itère sur les correspondances et affiche la portion de texte
for match_id, start, end in matches:
    print("Correspondance trouvée :", doc[start:end].text)

Nombre de correspondances trouvées : 10
Correspondance trouvée : Normandie
Correspondance trouvée : Stofflet
Correspondance trouvée : Cathelineau
Correspondance trouvée : Écosse
Correspondance trouvée : Rassurez
Correspondance trouvée : Allons
Correspondance trouvée : Fox
Correspondance trouvée : Neuville
Correspondance trouvée : volontiers
Correspondance trouvée : hasard


## Traitement massif des données

TODO documentation de l'utilisation de `nlp.pipe` plutôt à ce moment-là ?

Lorsqu'on souhaite traiter plusieurs documents, on peut utiliser `nlp.pipe(LISTE_DE_TEXTES)`.
Dans ce cas, on obtient une liste de documents en sortie, qu'il est possible d'inspecter avec une seconde boucle !

In [31]:
TEXTES = [
    "Bonjour tout le monde !", 
    "Comment allez-vous ? Bien, j'espère !",
    "Savez-vous qu'une chaîne de caractères peut contenir des retours à la ligne\ncomme celui-ci ?"
    ]
documents = nlp.pipe(TEXTES)
for doc_id, doc in enumerate(documents):
    for token in doc:
        print(f"doc#{doc_id}, tok#{token.i}: {token.text}")

doc#0, tok#0: Bonjour
doc#0, tok#1: tout
doc#0, tok#2: le
doc#0, tok#3: monde
doc#0, tok#4: !
doc#1, tok#0: Comment
doc#1, tok#1: allez
doc#1, tok#2: -vous
doc#1, tok#3: ?
doc#1, tok#4: Bien
doc#1, tok#5: ,
doc#1, tok#6: j'
doc#1, tok#7: espère
doc#1, tok#8: !
doc#2, tok#0: Savez
doc#2, tok#1: -vous
doc#2, tok#2: qu'
doc#2, tok#3: une
doc#2, tok#4: chaîne
doc#2, tok#5: de
doc#2, tok#6: caractères
doc#2, tok#7: peut
doc#2, tok#8: contenir
doc#2, tok#9: des
doc#2, tok#10: retours
doc#2, tok#11: à
doc#2, tok#12: la
doc#2, tok#13: ligne
doc#2, tok#14: 

doc#2, tok#15: comme
doc#2, tok#16: celui-ci
doc#2, tok#17: ?


In [32]:
%%time
# FIXME use the dataset file here
documents = nlp.pipe([load_text(path) for path in files])
per_tokens = []
spat_tokens = []
for doc in documents:
    for ent in doc.ents:
        if ent.label_ in ("PER"):
            per_tokens.append(ent)
        elif ent.label_ in ("LOC", "GPE"):
            spat_tokens.append(ent)
print(f"Got {len(per_tokens)} entites about persons, and {len(spat_tokens)} entities about spatial objects.")

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
Got 1655 entites about persons, and 1562 entities about spatial objects.


In [34]:
spat_tokens

[I,
 Norine Duclos,
 Adonc,
 Rose,
 Norine,
 Saint-Brunelle,
 Armandine,
 Rose,
 réserve de la Rose,
 Norine,
 couseuses,
 couseuses,
 Garde-les,
 Tiens,
 Norine,
 Bassette,
 bras-le-corps,
 Sorgues,
 Vaucluse,
 Château de Saint-Estève,
 Durance,
 notre Provence,
 Oasis,
 Saint-Estève,
 Durance,
 Notre voisine,
 Paris,
 Prologue,
 l'Amérique,
 l'Amérique,
 Amérique,
 Espagnols,
 les Incas,
 l'Europe,
 l'Amérique,
 États,
 État de Sonora,
 rio Gila,
 État,
 sierra Madre,
 golfe de Californie,
 sierra Madre,
 Durango,
 États de Durango,
 Guadalajara,
 Pacifique,
 Sonora,
 rio Gila,
 sierra Madre,
 Indiens,
 Comanches,
 Pawnees,
 Pimas,
 Opatas,
 Apaches,
 Sonora,
 Guaymas,
 Hermosillo,
 Pacifique,
 Montagne,
 Hermosillo,
 Gambusinos,
 Rosario,
 rue du Rosaire,
 Mexique,
 I,
 l'Amérique,
 la France,
 Canada,
 Nouvelle-France,
 Anglais,
 Angleterre,
 Mississipiens,
 la France,
 Canada,
 Européens,
 Indiens,
 Nouvelle-France,
 baie d'Hudson,
 golfe du Mexique,
 Nouvelle-Orléans,
 la France,

## Mise en place d'une évaluation objective
Très important : besoin d'avoir une référence validée, aussi appelée "vérité terrain" (*"ground truth"*), "données cibles" (*"targets"*), *"gold standard"*…

Cette référence contient, pour une échantillon représentatif de données d'entrée de notre système, les données idéales que notre système devrait produire en sortie.
Dans le doute, il est important de bien coller à la définition d'une tâche de traitement de données "classique", c'est à dire à un triplet (type et format des données d'entrées, type et format des données de sortie, méthode d'évaluation de la conformité entre données prédite et données attendues) communément utilisé par les équipes expérimentées sur ce sujet.

TODO introduire notions de precision/recall/fscore (métriques de détection / retrieval)

In [74]:
# On charge le dataset dans un format facile
import json
def load_dataset(path_to_json: str) -> dict[str, tuple[str, list[tuple[int, int, str]]]]:
    with open(path_to_json, encoding="utf8") as in_file:
        return json.load(in_file)

all_data = load_dataset("../dataset/French_ELTEC_NER_Open_Dataset.json")
print(f"Loaded text and target entities for {len(all_data)} samples.")

Loaded text and target entities for 100 samples.


In [75]:
from spacy.scorer import Scorer
from spacy.training.example import Example

def evaluate(ner_model, dataset_dict, debug=False):
    """FIXME DOC"""
    examples = []
    for doc_id, (text, target_entities) in dataset_dict.items():
        pred_doc = ner_model(text)
        if debug:
            print("Pred.:", [(ent.text, ent.label_) for ent in pred_doc.ents], " ↔ Targ.:", [(text[e[0]:e[1]], e[2]) for e in target_entities])
        try:
            example = Example.from_dict(pred_doc, {"entities": target_entities})
            examples.append(example)
        except ValueError as e:
            err_msg = f"Error parsing document '{doc_id}': "
            err_msg += getattr(e, "msg", str(e))
            print(err_msg)
            raise ValueError(err_msg)
    
    scorer = Scorer()
    scores = scorer.score_spans(examples, "ents")
    # print(scores["ents_f"])
    return scores

In [80]:
# Load a NER model
ner_model = spacy.load('fr_core_news_sm')

We should deactivate the useless parts of the pipeline here, to accelerate the evaluation.

In [81]:
ner_model.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [82]:
ner_model.select_pipes(enable="ner")
ner_model.pipe_names

['ner']

In [85]:
%%time
# evaluate using custom function, maybe useless because of the Language.evaluate() method! <https://spacy.io/api/language#evaluate>
results = evaluate(ner_model, all_data, debug=False)
results

/home/jchazalo/.virtualenvs/hn-ariane-ner-tuto-2023-PwR_0BG5/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "PREMIÈRE PARTIE -- LA CONSPIRATION EN DENTELLES
Où..." with entities "[[51, 58, 'PER'], [106, 113, 'PER'], [369, 374, 'L...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jchazalo/.virtualenvs/hn-ariane-ner-tuto-2023-PwR_0BG5/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "LE BANQUET
Dans la grande salle des fêtes de 1' « ..." with entities "[[1003, 1014, 'PER'], [1246, 1252, 'PER'], [1254, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jchazalo/.virt

CPU times: user 7.77 s, sys: 213 ms, total: 7.98 s
Wall time: 7.98 s


{'ents_p': 0.4317656129529684,
 'ents_r': 0.6339622641509434,
 'ents_f': 0.5136829231004433,
 'ents_per_type': {'MISC': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'PER': {'p': 0.6165458937198067,
   'r': 0.5984759671746777,
   'f': 0.6073765615704938},
  'LOC': {'p': 0.4205488194001276,
   'r': 0.698093220338983,
   'f': 0.5248904818797292},
  'ORG': {'p': 0.0, 'r': 0.0, 'f': 0.0}}}


Try the evaluation using the [`Language.evaluate()`](https://spacy.io/api/language#evaluate) method.

In [86]:
%%time
examples = []
for doc_id, (text, target_entities) in all_data.items():
    base_doc = ner_model.make_doc(text)  # We create simpler examples here but will the evaluate function recompute them?
    try:
        example = Example.from_dict(base_doc, {"entities": target_entities})
        examples.append(example)
    except ValueError as e:
        err_msg = f"Error parsing document '{doc_id}': "
        err_msg += getattr(e, "msg", str(e))
        print(err_msg)
        raise ValueError(err_msg)
print(f"Created {len(examples)} examples.")

/home/jchazalo/.virtualenvs/hn-ariane-ner-tuto-2023-PwR_0BG5/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "PREMIÈRE PARTIE -- LA CONSPIRATION EN DENTELLES
Où..." with entities "[[51, 58, 'PER'], [106, 113, 'PER'], [369, 374, 'L...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jchazalo/.virtualenvs/hn-ariane-ner-tuto-2023-PwR_0BG5/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "LE BANQUET
Dans la grande salle des fêtes de 1' « ..." with entities "[[1003, 1014, 'PER'], [1246, 1252, 'PER'], [1254, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jchazalo/.virt

Created 100 examples.
CPU times: user 3.13 s, sys: 2.09 ms, total: 3.13 s
Wall time: 3.14 s


In [87]:
%%time
scores = ner_model.evaluate(examples)
scores

CPU times: user 7.78 s, sys: 8.48 s, total: 16.3 s
Wall time: 16.4 s


{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.43243243243243246,
 'ents_r': 0.6339622641509434,
 'ents_f': 0.5141545524100996,
 'ents_per_type': {'LOC': {'p': 0.4216250799744082,
   'r': 0.698093220338983,
   'f': 0.5257279617072198},
  'MISC': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'PER': {'p': 0.6172914147521161,
   'r': 0.5984759671746777,
   'f': 0.6077380952380953},
  'ORG': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'speed': 9838.436375865644}

On obtient les mêmes valeurs, mais plus lentement ; probablement car on fait une évaluation plus large avec l'évaluation de la tokenization et de la vitesse en plus.

## TODO produire soi-même des données d'entraînement ou de test

jeu de test : besoin d'une quantité et d'une variété suffisantes pour que les résulats soient significatifs. Ce jeu de données ne peut pas contenir de données vues pendant l'entraînement.

jeu d'entraînement : généralement besoin d'une quantité plus importante pour permettre la stabilisation des paramètres statistiques d'un modèle.
Ces données doivent être suffisamment variées pour permettre de capturer les subtilités des données à traiter, et assez représentatives pour capturer en priorité les généralités.

Dans les 2 cas, il faut préparer :
- de exemples de données d'entrée pour le sytème (échantillons de textes)
- les sorties parfaites attendues pour ces données (dans le cas du NER, liste des entités — avec position et étiquette — à extraire)


TODO indiquer exemple procédure la plus basique possible :
- identifier groupe de textes à étiqueter
- les importer dans <https://tecoholic.github.io/ner-annotator/> et annoter
- exporter les données
- les convertir au format adapté

### 🤓 Pour aller plus loin
Questions à pointer (sans forcément y répondre car ça serait pour une autre formation/atelier) :
- quelles étiquettes/labels ?
- quelles règles suivre, comment gérer les ambiguïtés ?
- comment distribuer le travail ? Comment assurer la cohérence entre le travail des différents annotateurs ?
- Comment diffuser ton travail, le partager, quelle licence utiliser ?